In [1]:
pip install -U langchain langchain-community langchain-openai chromadb langchain-text-splitters pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

 ········


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
pdf_path = "Short Story.pdf" 
loader = PyPDFLoader(pdf_path)
documents = loader.load() 

In [4]:
# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
splits = splitter.split_documents(documents)
print(splits[1])
print(len(splits))

page_content='shorter novel), authors generally draw f rom a common pool of  literary 
techniques.  
Short story writers may define their works as part of the artistic and personal 
expression of the form. They may also attemp t to resist categorization by genre 
and fixed formation.  
Short stories have deep roots and the power of short fiction has been recognised 
in modern society for hundreds of years. The short form is, conceivably, more 
natural to us than longer forms. We are  drawn to short stories as the well -told 
story, and as  William Boyd , the award -winning British author and short story 
writer has said:  
[short stories] seem to a nswer something very deep in our nature as if, for the 
duration of its telling, something special has been created, some essence of our' metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2020-03-04T21:46:59+03:00', 'author': 'hp', 'moddate': '2020-03-04T21:46:59+03:00', 'source': 'Short Story.

In [ ]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
splits = splitter.split_documents(documents)

print("num chunks:", len(splits))
if splits:
    print(splits[0].page_content[:500])  # optional peek

# Imports must be on their own lines
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

num chunks: 15
Introduction to Short Story 
 
A short story is a piece of  prose fiction that typically can be read in one sitting 
and focuses on a self -contained incident or series of linked incidents, with the 
intent of evoking a "single effect" or mood.  
A dictionary definition is "an invented prose narrative shorter than a novel 
usually dealing with a few characters and aiming at unity of effect and often 
concentrating on the creation of mood rather than plot."[1]  
The short story is a crafted form 


In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Using only the information in <context>, write a brief news article.\n"
    "- Headline: one line, Title Case, no quotes.\n"
    "- Body: exactly three short paragraphs (2–4 sentences each).\n"
    "- Do not add facts that aren’t in the context.\n\n"
    "Topic: {topic}\n\n<context>\n{context}\n</context>"
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

def format_docs(retrieved_chunks):
    texts = [chunk.page_content for chunk in retrieved_chunks]
    return "\n\n".join(texts)

rag_chain = (
    {"context": retriever | format_docs, "topic": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
topic = "Important themes in the short story"  # or: input("Topic/angle: ").strip()
article = rag_chain.invoke(topic)
print(article)
